<a href="https://colab.research.google.com/github/darksister1331/ActionBoard-AI/blob/main/Screen_recording.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%
import os
import time
import subprocess
import signal
from datetime import datetime
import pygetwindow as gw

#%%
# CONFIGURATION
ZOOM_LINK = "zoom_link"
AUDIO_DEVICE_NAME = "Microphone Array (Intel® Smart Sound Technology for Digital Microphones)"
FFMPEG_PATH = r"location of file path"

#%%
# OUTPUT FILENAMES
recordings_dir = "dir_name"
os.makedirs(recordings_dir, exist_ok=True)

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
mkv_file = os.path.join(recordings_dir, f"zoom_meeting_{timestamp}.mkv")
mp4_file = os.path.join(recordings_dir, f"zoom_meeting_{timestamp}.mp4")
audio_file = os.path.join(recordings_dir, f"zoom_meeting_{timestamp}.mp3")  # use .wav if preferred

#%%
# LAUNCH ZOOM
print("Opening Zoom meeting...")
os.startfile(ZOOM_LINK)
print("Waiting 15 seconds for Zoom to launch...")
time.sleep(15)

#%%
# START FFMPEG RECORDING TO .MKV
print("Starting screen + audio recording to MKV...")

ffmpeg_command = [
    FFMPEG_PATH,
    "-y",
    "-f", "gdigrab",
    "-framerate", "30",
    "-i", "desktop",
    "-f", "dshow",
    "-i", f"audio={AUDIO_DEVICE_NAME}",
    "-vcodec", "libx264",
    "-acodec", "aac",
    "-pix_fmt", "yuv420p",
    "-preset", "ultrafast",
    mkv_file
]

recording_process = subprocess.Popen(ffmpeg_command, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP)

#%%
# FUNCTION TO CHECK IF ZOOM MEETING WINDOW IS OPEN
def is_zoom_meeting_window_open():
    return any("zoom meeting" in title.lower() for title in gw.getAllTitles())

#%%
# MONITOR ZOOM MEETING WINDOW
print("Recording... will stop automatically when Zoom meeting window closes.")
try:
    while True:
        if not is_zoom_meeting_window_open():
            print("Zoom meeting has ended. Stopping recording...")
            break
        time.sleep(5)
except KeyboardInterrupt:
    print("Recording interrupted manually.")
finally:
    recording_process.send_signal(signal.CTRL_BREAK_EVENT)
    try:
        recording_process.wait(timeout=10)
    except subprocess.TimeoutExpired:
        recording_process.kill()
    print(f" MKV Recording saved as: {os.path.abspath(mkv_file)}")

#%%
# CONVERT MKV TO MP4
print("Converting MKV to MP4...")
convert_command = [
    FFMPEG_PATH,
    "-i", mkv_file,
    "-c", "copy",
    "-movflags", "+faststart",
    mp4_file
]
subprocess.run(convert_command)
print(f" MP4 version saved as: {os.path.abspath(mp4_file)}")

#%%
# EXTRACT AUDIO FROM MKV TO MP3
print("Extracting audio to MP3...")
extract_audio_command = [
    FFMPEG_PATH,
    "-i", mkv_file,
    "-vn",                # Skip video
    "-acodec", "mp3",     # Use "pcm_s16le" if you prefer WAV
    audio_file
]
subprocess.run(extract_audio_command)
print(f" Audio file saved as: {os.path.abspath(audio_file)}")


